# Scrape Cost of Living Information From Numbeo.com

First, import the necessary libraries.

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

Create a list of all city names in Canada. This will be appended to the url to get a specific url for each cities.

In [2]:
# Make request
import requests
cities_requests = requests.get("https://www.numbeo.com/cost-of-living/country_result.jsp?country=Canada")

# Create BeautifulSoup object
from bs4 import BeautifulSoup
cities_soup = BeautifulSoup(cities_requests.text)

# Get list of city names
cities = [cities_soup.find("select").find_all("option")[j].text for j in range(len(cities_soup.find("select").find_all("option")))]

Minor adjustments to city names that will be appended to the url. 

In [3]:
# Adjust city names to be appended as url
cities = cities[1:]
cities = [city.replace(", ", "-").replace(" ","-") for city in cities]

cities_canada = cities.copy()
cities_canada = [city_canada+"-Canada" for city_canada in cities_canada]

cities = cities + cities_canada

From numbeo.com, scrape the cost of living for cities in Canada and present the results in a data frame. If there is no url for a specific city, it will be skipped.

In [4]:
costofliving = pd.DataFrame()
for city in cities:
    try:
        # Make request
        col_requests = requests.get("https://www.numbeo.com/cost-of-living/in/"+city)

        # Create BeautifulSoup object
        col = BeautifulSoup(col_requests.text)

        # Get city and country
        try:
            city, country = col.find("span", {"class":"purple_light"}).text.split(",")
        except:
            city, country = city, "Canada"

        # Find the living cost table
        table = col.find("table", {"class":"data_wide_table new_bar_table"})

        # Get item and cost
        table_item_cost = table.find_all("td")
        item = list()
        cost = list()
        for i in range(len(table_item_cost)):
            if i % 3 == 0:
                item.append(table_item_cost[i].text)
            if i % 3 == 1:
                cost.append(float(table_item_cost[i].text.split()[0].replace(",","")))

        # Store result in dataframe
        df1 = pd.DataFrame({'city':[city], 'country':[country]})
        df2 = pd.DataFrame({item[j]:[cost[j]] for j in range(len(item))})
        df = pd.concat([df1,df2], axis=1)
        costofliving = pd.concat([costofliving,df], axis=0)

    except:
        pass

    time.sleep(.1)

costofliving = costofliving.reset_index(drop=True)

Some city names can be found outside of Canada (e.g. London, UK), so here I removed rows which country is not Canada.

In [5]:
# Remove countries that is not Canada
costofliving['country'] = [row.strip().replace('ON', 'Canada').replace('BC','Canada') for row in costofliving['country']]
costofliving = costofliving[costofliving['country'] == 'Canada']
costofliving['city'] = [row.strip() for row in costofliving['city']]
costofliving = costofliving.drop_duplicates(subset=['city'])

Write the results to a csv file.

In [6]:
costofliving.to_csv("costofliving.csv", index=False)